In [1]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
import numpy as np
from keras import backend as K
from keras.layers import Layer, Input
from skimage.feature import greycomatrix
import tensorflow as tf

class GLCM(Layer):


    def __init__(self, BATCH_SIZE = 100, d = [1], theta = [0], levels = 256, symmetric = True, normed = True, **kwargs ):
        super(GLCM, self).__init__(**kwargs)
        self.d = d
        self.theta = theta
        self.levels = levels
        self.symmetric = symmetric
        self.normed = normed
        self.batch_size = BATCH_SIZE

    def compute_output_shape(self, input_shape):

        return (self.batch_size, 4)

    def binarization(self, x):
        bi_x = K.sign(x)
        return bi_x
    
    def call(self, inputs):
        #make input becomes a binarized image
        image = self.binarization(inputs)

        #calculate GLCM(in a batch shape)
        a = image[:, :, :-1, :]
        b = image[:, :, 1:, :]

        a_or_b = K.maximum(a, b)
        a_and_b = a*b

        one = tf.ones(shape = a.shape)

        not_a = one-a
        not_b = one-b

        # not(a or b)
        M0_0 = K.map_fn(K.sum, (one-a_or_b))

        # b and (not a)
        M0_1 = K.map_fn(K.sum, (b*(not_a)))

        # a and (not b)
        M1_0 = K.map_fn(K.sum, (a*(not_b)))

        # a and b
        M1_1 = K.map_fn(K.sum, (a_and_b))


        output = tf.stack([M0_0, M0_1, M1_0, M1_1], axis = 1)

        
        #print("output shape: ", output.shape)
        
        return output

In [3]:
x_train = np.random.randint(2, size=(1, 28, 28, 1))

In [4]:
x = Input(batch_shape=(1, 28, 28, 1))
y = GLCM()(x)

In [5]:
result = K.function([x], [y])([x_train])
result

[array([[ 201.,  193.,  192.,  170.]], dtype=float32)]

In [6]:
result = greycomatrix(x_train.reshape([28, 28]),
                      [1],
                      [0],
                      levels = 2,
                      symmetric = False,
                      normed = False)
result

array([[[[201]],

        [[193]]],


       [[[192]],

        [[170]]]], dtype=uint32)